In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/food-ingredients-and-allergens/food_ingredients_and_allergens.csv")

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.drop_duplicates(inplace = True)
df.isna().sum()

In [ ]:
df.dropna(inplace = True)
df.shape

In [ ]:
for item in list(df.columns):
    print(item , "   " , df[item].nunique())

# Similarity

In [ ]:
df[(df['Allergens'] == 'None')&(df['Prediction']=="Contains")]

No dissimilarities above 

In [ ]:
X = df.drop(['Prediction','Allergens'], axis  =1)
y = pd.DataFrame(df['Prediction'])

In [ ]:
for column in list(X.columns):
    X[column+' '+'Label'] = X[column].map(dict(zip(pd.Series(X[column].unique()),pd.Series([i for i in range(X[column].nunique())]))))
    

In [ ]:
X.drop(['Food Product', 'Main Ingredient', 'Sweetener', 'Fat/Oil', 'Seasoning'], axis =1 , inplace = True)
X.head()

In [ ]:
X.head()

In [ ]:
y['Prediction Label'] = y['Prediction'].map({"Contains":1,"Does not contain" : 0})
y.drop(['Prediction'], axis =1 , inplace =True)
y.head()

In [ ]:
from sklearn.model_selection import train_test_split

X_train , X_test , y_train , y_test = train_test_split(X,y , test_size=0.2 , random_state=23, stratify=y)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Input(shape=(X_train_scaled.shape[1],)),  # Input layer
    layers.Dense(128, activation='relu'),  # First hidden layer
    layers.Dense(64, activation='relu'),   # Second hidden layer
    layers.Dense(1, activation='sigmoid')   # Output layer
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train_scaled, y_train, epochs=50, batch_size=8, validation_data=(X_test_scaled, y_test))

In [ ]:
model.save('fooda1model.h5')

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test)
print(f"Test Accuracy: {test_accuracy:.4f}")